## Уграђивања

У претходном примеру радили смо са векторима високе димензије заснованим на моделу "вреће речи" дужине `vocab_size`, и експлицитно смо конвертовали векторе нискодимензионалне позиционе репрезентације у ретке једнодимензионалне репрезентације. Ова једнодимензионална репрезентација није ефикасна у погледу меморије. Поред тога, свака реч се третира независно од других, тако да једнодимензионални кодирани вектори не изражавају семантичке сличности између речи.

У овој јединици наставићемо да истражујемо скуп података **News AG**. За почетак, учитаћемо податке и преузети неке дефиниције из претходне јединице.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Шта је уграђивање?

Идеја **уграђивања** је да се речи представе помоћу густих вектора нижих димензија који одражавају семантичко значење речи. Касније ћемо разговарати о томе како изградити смислена уграђивања речи, али за сада замислимо уграђивања као начин да се смањи димензионалност векторa речи.

Дакле, слој за уграђивање узима реч као улаз и производи излазни вектор одређене величине `embedding_size`. У неком смислу, веома је сличан слоју `Dense`, али уместо да узима једновекторски кодирани улаз, он може да прихвати број речи.

Коришћењем слоја за уграђивање као првог слоја у нашој мрежи, можемо прећи са модела „вреће речи“ на модел **вреће уграђивања**, где прво претварамо сваку реч у нашем тексту у одговарајуће уграђивање, а затим израчунавамо неку агрегатну функцију над свим тим уграђивањима, као што су `sum`, `average` или `max`.

![Слика која приказује класификатор са уграђивањем за пет речи у низу.](../../../../../translated_images/sr/embedding-classifier-example.b77f021a7ee67eee.webp)

Наша неуронска мрежа класификатора састоји се од следећих слојева:

* Слој `TextVectorization`, који узима стринг као улаз и производи тензор бројева токена. Одредићемо разумну величину речника `vocab_size` и игнорисати речи које се ређе користе. Улазни облик ће бити 1, а излазни облик ће бити $n$, јер ћемо добити $n$ токена као резултат, при чему сваки од њих садржи бројеве од 0 до `vocab_size`.
* Слој `Embedding`, који узима $n$ бројева и смањује сваки број на густ вектор одређене дужине (100 у нашем примеру). Тако ће улазни тензор облика $n$ бити трансформисан у тензор облика $n\times 100$.
* Агрегациони слој, који узима просек овог тензора дуж прве осе, тј. израчунава просек свих $n$ улазних тензора који одговарају различитим речима. За имплементацију овог слоја користићемо слој `Lambda` и у њега проследити функцију за израчунавање просека. Излаз ће имати облик 100 и представљаће нумеричку репрезентацију целокупног улазног низа.
* Завршни `Dense` линеарни класификатор.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


У `summary` испису, у колони **output shape**, прва димензија тензора `None` одговара величини мини-бача, а друга дужини секвенце токена. Све секвенце токена у мини-бачу имају различите дужине. Разговараћемо о томе како се носити с тим у наредном одељку.

Сада хајде да обучимо мрежу:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


**Напомена** да градимо векторизатор на основу подскупа података. Ово се ради како би се убрзао процес, и то може довести до ситуације у којој нису сви токени из нашег текста присутни у речнику. У том случају, ти токени би били игнорисани, што може резултирати нешто нижом тачношћу. Међутим, у стварном животу подскуп текста често даје добру процену речника.


### Рад са променљивим величинама секвенци

Хајде да разумемо како се обука одвија у мини-бачовима. У горњем примеру, улазни тензор има димензију 1, и користимо мини-бачове дужине 128, тако да је стварна величина тензора $128 \times 1$. Међутим, број токена у свакој реченици је различит. Ако применимо слој `TextVectorization` на један улаз, број враћених токена је различит, у зависности од тога како је текст токенизован:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Међутим, када применимо векторизатор на неколико секвенци, он мора да произведе тензор правоугаоног облика, па попуњава неискоришћене елементе са PAD токеном (који је у нашем случају нула):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Овде можемо видети уградње:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Напомена**: Да би се минимизирала количина попуњавања, у неким случајевима има смисла сортирати све секвенце у датасету по редоследу растуће дужине (или, прецизније, броја токена). Ово ће осигурати да сваки минибатч садржи секвенце сличне дужине.


## Семантичке уградње: Word2Vec

У нашем претходном примеру, слој за уградњу је научио да мапира речи у векторске репрезентације, али те репрезентације нису имале семантичко значење. Било би корисно научити векторску репрезентацију тако да сличне речи или синоними одговарају векторима који су блиски једни другима у смислу неке векторске удаљености (на пример, еуклидске удаљености).

Да бисмо то постигли, потребно је да претходно обучимо наш модел за уградњу на великој колекцији текста користећи технику као што је [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ова техника се заснива на две главне архитектуре које се користе за производњу расподељене репрезентације речи:

 - **Континуална торба речи** (CBoW), где тренирамо модел да предвиди реч из околног контекста. Дат је нграм $(W_{-2},W_{-1},W_0,W_1,W_2)$, циљ модела је да предвиди $W_0$ из $(W_{-2},W_{-1},W_1,W_2)$.
 - **Континуални скип-грам** је супротан од CBoW. Модел користи околни прозор контекстуалних речи да би предвидео тренутну реч.

CBoW је бржи, док је скип-грам спорији, али боље представља ретке речи.

![Слика која приказује алгоритме CBoW и Skip-Gram за конвертовање речи у векторе.](../../../../../translated_images/sr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Да бисмо експериментисали са Word2Vec уградњом претходно обученом на Google News скупу података, можемо користити библиотеку **gensim**. Испод налазимо речи које су најсличније 'neural'.

> **Напомена:** Када први пут креирате векторе речи, њихово преузимање може потрајати!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Можемо такође извући векторско уграђивање из речи, које ће се користити у обуци модела за класификацију. Уграђивање има 300 компоненти, али овде приказујемо само првих 20 компоненти вектора ради прегледности:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Сјајна ствар код семантичких угњежђивања је то што можете манипулисати кодирањем вектора на основу семантике. На пример, можемо тражити реч чија је векторска репрезентација што је могуће ближа речима *краљ* и *жена*, а што је могуће даља од речи *човек*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Пример изнад користи неку унутрашњу GenSym магију, али основна логика је заправо прилично једноставна. Занимљива ствар код уграђивања је да можете изводити нормалне операције са векторима на векторима уграђивања, и то би одражавало операције на значењима речи. Пример изнад може се изразити у терминима векторских операција: израчунавамо вектор који одговара **КРАЉ-ЧОВЕК+ЖЕНА** (операције `+` и `-` се изводе на векторским представама одговарајућих речи), а затим проналазимо најближу реч у речнику том вектору:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **НАПОМЕНА**: Морали смо да додамо мале коефицијенте на *man* и *woman* векторе - пробајте да их уклоните да видите шта ће се десити.

Да бисмо пронашли најближи вектор, користимо TensorFlow механизам за израчунавање вектора растојања између нашег вектора и свих вектора у речнику, а затим проналазимо индекс минималне речи користећи `argmin`.


Иако Word2Vec изгледа као одличан начин за изражавање семантике речи, има много недостатака, укључујући следеће:

* И CBoW и skip-gram модели су **предиктивни уградни вектори**, и узимају у обзир само локални контекст. Word2Vec не користи предности глобалног контекста.
* Word2Vec не узима у обзир **морфологију речи**, односно чињеницу да значење речи може зависити од различитих делова речи, као што је корен.

**FastText** покушава да превазиђе друго ограничење и надограђује Word2Vec тако што учи векторске репрезентације за сваку реч и n-граме карактера који се налазе унутар сваке речи. Вредности репрезентација се затим просечавају у један вектор на сваком кораку тренинга. Иако ово додаје много додатних рачунања током претренинга, омогућава уградним векторима речи да кодирају информације о подречима.

Други метод, **GloVe**, користи другачији приступ уградним векторима речи, заснован на факторизацији матрице контекста речи. Прво, гради велику матрицу која броји број појављивања речи у различитим контекстима, а затим покушава да представи ову матрицу у нижим димензијама на начин који минимизира губитак реконструкције.

Библиотека gensim подржава те уградне векторе речи, и можете експериментисати са њима тако што ћете променити код за учитавање модела изнад.


## Коришћење претходно обучених уграђивања у Keras-у

Можемо изменити горњи пример како бисмо унапред попунили матрицу у нашем слоју за уграђивање семантичким уграђивањима, као што је Word2Vec. Речници претходно обученог уграђивања и текстуалног корпуса вероватно се неће поклапати, па морамо изабрати један. Овде истражујемо две могуће опције: коришћење речника токенизатора и коришћење речника из Word2Vec уграђивања.

### Коришћење речника токенизатора

Када користимо речник токенизатора, неке речи из речника ће имати одговарајућа Word2Vec уграђивања, а неке ће недостајати. С обзиром на то да је величина нашег речника `vocab_size`, а дужина вектора Word2Vec уграђивања `embed_size`, слој за уграђивање ће бити представљен тежинском матрицом облика `vocab_size`$\times$`embed_size`. Ову матрицу ћемо попунити пролазећи кроз речник:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


За речи које нису присутне у Word2Vec речнику, можемо их оставити као нуле или генерисати случајни вектор.

Сада можемо дефинисати слој за уграђивање са унапред обученим тежинама:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Напомена**: Обратите пажњу да смо поставили `trainable=False` приликом креирања `Embedding`, што значи да нећемо поново тренирати слој Embedding. Ово може довести до благо смањене тачности, али убрзава процес тренирања.

### Коришћење вокабулара за уграђивање

Један од проблема са претходним приступом је то што су вокабулари који се користе у TextVectorization и Embedding различити. Да бисмо превазишли овај проблем, можемо користити једно од следећих решења:
* Поново тренирати Word2Vec модел на нашем вокабулару.
* Учитати наш скуп података са вокабуларом из претходно тренираног Word2Vec модела. Вокабулари који се користе за учитавање скупа података могу се специфицирати током учитавања.

Други приступ изгледа једноставније, па хајде да га имплементирамо. Пре свега, креираћемо слој `TextVectorization` са специфицираним вокабуларом, преузетим из Word2Vec уграђивања:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Генсим библиотека за угњежђивање речи садржи погодну функцију, `get_keras_embeddings`, која ће вам аутоматски креирати одговарајући слој за угњежђивање у Керасу.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Један од разлога зашто не видимо већу тачност је то што неке речи из нашег скупа података недостају у претходно обученом GloVe речнику, и стога се у суштини игноришу. Да бисмо то превазишли, можемо обучити сопствене уградње засноване на нашем скупу података.


## Контекстуални уграђени прикази

Једно од главних ограничења традиционалних унапред обучених уграђених приказа, као што је Word2Vec, јесте чињеница да, иако могу да ухвате одређено значење речи, не могу да разликују различита значења. Ово може изазвати проблеме у моделима који се користе након тога.

На пример, реч 'play' има различита значења у следећим реченицама:
- Ишао сам на **представу** у позоришту.
- Џон жели да се **игра** са својим пријатељима.

Унапред обучени уграђени прикази о којима смо говорили представљају оба значења речи 'play' у истом уграђеном приказу. Да бисмо превазишли ово ограничење, потребно је да изградимо уграђене приказе засноване на **језичком моделу**, који је обучен на великом корпусу текста и *зна* како се речи могу комбиновати у различитим контекстима. Разматрање контекстуалних уграђених приказа је ван домета овог туторијала, али ћемо се вратити на њих када будемо говорили о језичким моделима у наредној јединици.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
